# CS490 Report

| Classifiers                | Score on Training | Score on Validation | Score on Leaderboard |
|----------------------------|-------------------|---------------------|----------------------|
| KNN (20)                   | 94.8%             | 89.8%               | 62.8%                |
| SVM (sigmoid)              | 82.04%            | 84.6%               | 82.6%                |
| Neural Network(15 neurons) | 72.4%             | 91.8%               | 81.2%                |